In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 53.5 MB/s 


In [3]:
from bs4 import BeautifulSoup as bs #웹 크롤링을 위한 모듈
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from konlpy.tag import Twitter
from google.colab import drive
import urllib.request #웹에 접근하기 위한 모듈
import pandas as pd
import ast
import os
import warnings
import re
import time


warnings.filterwarnings(action='ignore')
drive.mount('/content/drive')

# Pdf 전처리한 Text Data가 저장되어 있는 경로 
path = '/content/drive/MyDrive/Colab Notebooks/Mirae/Text_data/'

Mounted at /content/drive


#### 함수 / 불용어 정의

In [ ]:
stop_keyword = """
아
휴
아이구
아이쿠
아이고
어
나
우리
저희
따라
의해
을
를
에
의
가
으로
로
에게
뿐이다
의거하여
근거하여
입각하여
기준으로
예하면
예를 들면
예를 들자면
저
소인
소생
저희
지말고
하지마
하지마라
다른
물론
또한
그리고
비길수 없다
해서는 안된다
뿐만 아니라
만이 아니다
만은 아니다
막론하고
관계없이
그치지 않다
그러나
그런데
하지만
든간에
논하지 않다
따지지 않다
설사
비록
더라도
아니면
만 못하다
하는 편이 낫다
불문하고
향하여
향해서
향하다
쪽으로
틈타
이용하여
타다
오르다
제외하고
이 외에
이 밖에
하여야
비로소
한다면 몰라도
외에도
이곳
여기
부터
기점으로
따라서
할 생각이다
하려고하다
이리하여
그리하여
그렇게 함으로써
하지만
일때
할때
앞에서
중에서
보는데서
으로써
로써
까지
해야한다
일것이다
반드시
할줄알다
할수있다
할수있어
임에 틀림없다
한다면
등
등등
제
겨우
단지
다만
할뿐
딩동
댕그
대해서
대하여
대하면
훨씬
얼마나
얼마만큼
얼마큼
남짓
여
얼마간
약간
다소
좀
조금
다수
몇
얼마
지만
하물며
또한
그러나
그렇지만
하지만
이외에도
대해 말하자면
뿐이다
다음에
반대로
반대로 말하자면
이와 반대로
바꾸어서 말하면
바꾸어서 한다면
만약
그렇지않으면
까악
툭
딱
삐걱거리다
보드득
비걱거리다
꽈당
응당
해야한다
에 가서
각
각각
여러분
각종
각자
제각기
하도록하다
와
과
그러므로
그래서
고로
한 까닭에
하기 때문에
거니와
이지만
대하여
관하여
관한
과연
실로
아니나다를가
생각한대로
진짜로
한적이있다
하곤하였다
하
하하
허허
아하
거바
와
오
왜
어째서
무엇때문에
어찌
하겠는가
무슨
어디
어느곳
더군다나
하물며
더욱이는
어느때
언제
야
이봐
어이
여보시오
흐흐
흥
휴
헉헉
헐떡헐떡
영차
여차
어기여차
끙끙
아야
앗
아야
콸콸
졸졸
좍좍
뚝뚝
주룩주룩
솨
우르르
그래도
또
그리고
바꾸어말하면
바꾸어말하자면
혹은
혹시
답다
및
그에 따르는
때가 되어
즉
지든지
설령
가령
하더라도
할지라도
일지라도
지든지
몇
거의
하마터면
인젠
이젠
된바에야
된이상
만큼	어찌됏든
그위에
게다가
점에서 보아
비추어 보아
고려하면
하게될것이다
일것이다
비교적
좀
보다더
비하면
시키다
하게하다
할만하다
의해서
연이서
이어서
잇따라
뒤따라
뒤이어
결국
의지하여
기대여
통하여
자마자
더욱더
불구하고
얼마든지
마음대로
주저하지 않고
곧
즉시
바로
당장
하자마자
밖에 안된다
하면된다
그래
그렇지
요컨대
다시 말하자면
바꿔 말하면
즉
구체적으로
말하자면
시작하여
시초에
이상
허
헉
허걱
바와같이
해도좋다
해도된다
게다가
더구나
하물며
와르르
팍
퍽
펄렁
동안
이래
하고있었다
이었다
에서
로부터
까지
예하면
했어요
해요
함께
같이
더불어
마저
마저도
양자
모두
습니다
가까스로
하려고하다
즈음하여
다른
다른 방면으로
해봐요
습니까
했어요
말할것도 없고
무릎쓰고
개의치않고
하는것만 못하다
하는것이 낫다
매
매번
들
모
어느것
어느
로써
갖고말하자면
어디
어느쪽
어느것
어느해
어느 년도
라 해도
언젠가
어떤것
어느것
저기
저쪽
저것
그때
그럼
그러면
요만한걸
그래
그때
저것만큼
그저
이르기까지
할 줄 안다
할 힘이 있다
너
너희
당신
어찌
설마
차라리
할지언정
할지라도
할망정
할지언정
구토하다
게우다
토하다
메쓰겁다
옆사람
퉤
쳇
의거하여
근거하여
의해
따라
힘입어
그
다음
버금
두번째로
기타
첫번째로
나머지는
그중에서
견지에서
형식으로 쓰여
입장에서
위해서
단지
의해되다
하도록시키다
뿐만아니라
반대로
전후
전자
앞의것
잠시
잠깐
하면서
그렇지만
다음에
그러한즉
그런즉
남들
아무거나
어찌하든지
같다
비슷하다
예컨대
이럴정도로
어떻게
만약
만일
위에서 서술한바와같이
인 듯하다
하지 않는다면
만약에
무엇
무슨
어느
어떤
아래윗
조차
한데
그럼에도 불구하고
여전히
심지어
까지도
조차도
하지 않도록
않기 위하여
때
시각
무렵
시간
동안
어때
어떠한
하여금
네
예
우선
누구
누가 알겠는가
아무도
줄은모른다
줄은 몰랏다
하는 김에
겸사겸사
하는바
그런 까닭에
한 이유는
그러니
그러니까
때문에
그
너희
그들
너희들
타인
것
것들
너
위하여
공동으로
동시에
하기 위하여
어찌하여
무엇때문에
붕붕
윙윙
나
우리
엉엉
휘익
윙윙
오호
아하
어쨋든
만 못하다	하기보다는
차라리
하는 편이 낫다
흐흐
놀라다
상대적으로 말하자면
마치
아니라면
쉿
그렇지 않으면
그렇지 않다면
안 그러면
아니었다면
하든지
아니면
이라면
좋아
알았어
하는것도
그만이다
어쩔수 없다
하나
일
일반적으로
일단
한켠으로는
오자마자
이렇게되면
이와같다면
전부
한마디
한항목
근거로
하기에
아울러
하지 않도록
않기 위해서
이르기까지
이 되다
로 인하여
까닭으로
이유만으로
이로 인하여
그래서
이 때문에
그러므로
그런 까닭에
알 수 있다
결론을 낼 수 있다
으로 인하여
있다
어떤것
관계가 있다
관련이 있다
연관되다
어떤것들
에 대해
이리하여
그리하여
여부
하기보다는
하느니
하면 할수록
운운
이러이러하다
하구나
하도다
다시말하면
다음으로
에 있다
에 달려 있다
우리
우리들
오히려
하기는한데
어떻게
어떻해
어찌됏어
어때
어째서
본대로
자
이
이쪽
여기
이것
이번
이렇게말하자면
이런
이러한
이와 같은
요만큼
요만한 것
얼마 안 되는 것
이만큼
이 정도의
이렇게 많은 것
이와 같다
이때
이렇구나
것과 같이
끼익
삐걱
따위
와 같은 사람들
부류의 사람들
왜냐하면
중의하나
오직
오로지
에 한하다
하기만 하면
도착하다
까지 미치다
도달하다
정도에 이르다
할 지경이다
결과에 이르다
관해서는
여러분
하고 있다
한 후
혼자
자기
자기집
자신
우에 종합한것과같이
총적으로 보면
총적으로 말하면
총적으로
대로 하다
으로서
참
그만이다
할 따름이다
쿵
탕탕
쾅쾅
둥둥
봐
봐라
아이야
아니
와아
응
아이
참나
년
월
일
령
영
일
이
삼
사
오
육
륙
칠
팔
구
이천육
이천칠
이천팔
이천구
하나
둘
셋
넷
다섯
여섯
일곱
여덟
아홉
령
영
노
통
부
제
조
항
등
누구
상태
법
규정
외
대한
뿐
그
호
"""
stop_keyword = stop_keyword.replace('\n', ' ')
stop_keyword_lst = stop_keyword.split()

In [4]:
# 테이블이 제거된 text에서 전처리 과정에서 소실된 단어들로 인해 노이즈 데이터가 생성 그리고 text의 정성적인 부분만 보기 위해서 
def clean_text(text):
    text = text.replace(".", "").strip()
    text = text.replace("·", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    return text

In [5]:
# esg 포털에서 21년 esg 등급 크롤링 함수 
def page_list(page):
    ESGLIST_URL = "http://www.cgs.or.kr/business/esg_tab04.jsp?pg={}&pp=10&skey=&svalue=&sfyear=2021&styear=2021&sgtype=&sgrade=#ui_contents".format(page) #주소설정
    response = urllib.request.urlopen(ESGLIST_URL)
    ESGLIST_HTML = response.read()

    soup = bs(ESGLIST_HTML)
    STK_ESG_GRADE=pd.DataFrame()
    if page <= 100:
        num_list = [0,1,2,3,4,5,6,7,8,9]
    else:
        num_list = [0,1,2,3,4]
    for i in num_list:
        find_stk = soup.find_all('td')[i*9:i*9+9]
        stk_col_list=[]
        for i in [0,1,2,3,4,5,6,7,8]:
            stk_col_list.append(find_stk[i].text)
            clean_stk_col_list=pd.DataFrame(stk_col_list).transpose()
        STK_ESG_GRADE = pd.concat([STK_ESG_GRADE,clean_stk_col_list])
    return STK_ESG_GRADE

In [6]:
# 불용어 처리, 토큰화 함수 
def tokenize(doc) : 
    post_tagger = Twitter( )
    noun_list = post_tagger.nouns(clean_text(doc))
    final_noun_list = [''.join(t) for t in noun_list if t not in stop_keyword]
    return final_noun_list

In [7]:
# lda 모델링의 결과물인 키워드를 리스트에 저장하고 print해주는 함수 
def save_topics(model, feature_names, no_top_words, lst) :
    for topic in model.components_ :
        
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes = topic_word_indexes[:no_top_words]
        
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])
        lst.append(feature_concat)

#### Train Data: PDF에서 테이블 추출 전처리 

In [8]:
Text_df=pd.read_csv(path+"21Y_total_report.csv")
# df에 str형태로 전환이 되어서 저장이 되었기 때문에 str->list 그리고 그 안의 dict 형태로 읽기위해 변환
for i in range(len(Text_df)):
  Text_df['Text'].iloc[i] = ast.literal_eval(Text_df['Text'].iloc[i])
  
report_name_list = list(Text_df['Name'])

In [9]:
final_result_dict = {}
# pdf에서 모든 내용을 다 추출한 후 그 원본 결과를 이용해서 해당 데이터에 pdf에 존재하는 table을 제거하는 과정 
# 1차적으로 추출한 데이터의 개행문자가 보통 테이블 형식에서 많이 나오는 것을 발견 개행 문자가 나타나는 주기를 계산하여 평균보다 낮으면 테이블이라 판단 제거하는 코드 
for i in range(len(report_name_list)):

    #print("파일명: ",report_name_list[i])
    # 각 페이지의 내용을 \n나 \n\n 기준으로 split하여 담은 단어들의 리스트, page_word
    page_word = []
    word_len = []
    page_word_len = []
    result_list = []
    page_text = Text_df['Text'].iloc[i][0]

    # 보고서의 전체 페이지를 돌면서 \n나 \n\n 기준으로 나눠진 문장들을 page_word 리스트에 담아준다
    # re.split하면서 빈 문자열('')이 page_word 리스트의 요소로 들어가는데, 이는 불필요하므로 page_word 리스트에 담지 않도록 한다
    # 추후 단어들의 길이를 통해, 해당 단어가 문장(긴 단어의 경우 문자로 판단할 예정)인지, 표의 일부인지를 확인해야 하므로, 단어(word)의 길이도 수집한다
    for page_num in range(len(page_text)):
      page_word.append([])
      page_word_len.append([])
      split_temp = re.split('[\n|\n\n]', page_text[page_num])
      for word in split_temp:
          if word != '':
              page_word[page_num].append(word) 
              page_word_len[page_num].append(len(word))
              word_len.append(len(word))
    mean_word = sum(word_len)/len(word_len)

    for page_num in range(len(page_text)):
        page_result = []
        for index in range(len(page_word_len[page_num])):
            len_w = page_word_len[page_num][index]
            if len_w > mean_word:
                page_result.append(page_word[page_num][index])
        result_list.append(page_result)
    final_result_dict[report_name_list[i]]=result_list

In [10]:
# 테이블 추출을 위해서 리스트에 끊긴 단위로 넣었기 때문에 리스트 안 str을 서로 붙여줘야한다.
final_sumtext_dict = {}
for i in range(len(report_name_list)):
  file_name = report_name_list[i]
  text_list = final_result_dict[file_name]
  result_str = ''
  for texts in text_list:
    for sen in texts:
      result_str += sen
  result_str = clean_text(result_str)
  final_sumtext_dict[file_name] = result_str

In [11]:
pre_text = pd.Series(final_sumtext_dict.values())
Text_df['Text']= pre_text

In [12]:
#  ESG 크롤링 후 데이터 프레임 병합
df_esg = pd.DataFrame()
for i in tqdm_notebook(list(range(1,101))):
    df_temp = page_list(i)
    df_esg = pd.concat([df_esg,df_temp])
    df_esg

df_esg.columns = ['NO', '기업명','기업코드','ESG등급','환경','사회','지배구조','평가년도','비고']
df_esg.reset_index(inplace=True)
df_esg.drop(columns=['index','NO','비고'], inplace=True)
df_esg.head()

  0%|          | 0/100 [00:00<?, ?it/s]

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,AJ네트웍스,095570,B,D,B,B,2021
1,AK홀딩스,006840,B+,B,B+,B+,2021
2,AP시스템,265520,D,D,D,C,2021
3,BGF,027410,A,A,A+,A,2021
4,BGF리테일,282330,A,A,A+,A,2021


In [13]:
index = 0
Text_df['E_G'] = ''
Text_df['S_G'] = ''
Text_df['G_G'] = ''
Text_df['ESG_G'] = ''

# 기업명을 맞춰주기 위해 현재 파일명으로 되어있는 것을 기업명이 되도록 전처리 
# 기업명에 맞게 esg 등급 할당 
for r_name in report_name_list:
  if r_name.endswith('_kr.pdf'):
    com_name = r_name.replace('_kr.pdf', '')
  elif r_name.endswith('_esg.pdf'):
    com_name = r_name.replace('_esg.pdf', '')
  else:
    com_name = r_name.replace('.pdf', '')
  for i in range(len(df_esg)):
    if com_name == df_esg['기업명'].iloc[i]:
      ESG_score = str(df_esg['ESG등급'].iloc[i])
      E_score = str(df_esg['환경'].iloc[i])
      S_score = str(df_esg['사회'].iloc[i])
      G_score = str(df_esg['지배구조'].iloc[i])


  Text_df['ESG_G'].iloc[index] = ESG_score
  Text_df['E_G'].iloc[index] = E_score
  Text_df['S_G'].iloc[index] = S_score
  Text_df['G_G'].iloc[index] = G_score
  index += 1

In [14]:
# Total ESG별로 기업들의 Text를 모으기 위해 즉, 등급별 pool을 만들기 위해 dict 생성 후 ESG등급별로 Text 할당
G_list = list(Text_df['ESG_G'].unique())
ESG_G_n_text_dict = {}

for i in G_list:
  ESG_G_n_text_dict[i] = ''

for i1 in range(len(Text_df)):
  ESG_G = Text_df['ESG_G'].iloc[i1]
  new_str = ESG_G_n_text_dict[ESG_G] + Text_df['Text'].iloc[i1]
  ESG_G_n_text_dict[ESG_G] = new_str

#### 명사를 추출하기 위해 위에서 정의한 토큰화 함수와 불필요한 조사,  한글 불용어(stopwords) 리스트를 활용해서 Text 토큰화 진행 

In [17]:
# ESG_G_n_text_dict에 있는 등급별 text를 사전에 정의한 토큰화 함수를 사용해서 토큰화 
for G in ESG_G_n_text_dict.keys():
  print("진행중인 등급: ",G)
  result = tokenize(ESG_G_n_text_dict[G]) 
  ESG_G_n_text_dict[G] = result

진행중인 등급:  A+
진행중인 등급:  B
진행중인 등급:  B+
진행중인 등급:  A


In [53]:
Result_G_n_text_dict = {}
Result_G_n_text_lda_dict = {}

# 토큰화된 결과물을 TfidfVectorizer를 사용해서 벡터화 후 lDA모델링 실시
# ngram_range=(1,2),max_df=500 은 약 200개의 실험을 진행 후 가장 보편적으로 잘 나오는 값을 할당
# 보고서의 Text를 total_esg 기준으로 나누었기 때문에 해당 보고서의 토픽을 1개라고 기준을 잡음
# 따라서 n_components =1로 할당 

for G in ESG_G_n_text_dict.keys():
  print("진행중인 등급: ",G)
  # Tfidfvectorizer 변환
  tfidf_vect=TfidfVectorizer(ngram_range=(1,2),max_df=500)
  Result_G_n_text_dict[G]=tfidf_vect
  tfidf_vect=tfidf_vect.fit_transform(ESG_G_n_text_dict[G])
  
  # lda 모델에 입력 
  lda_train = LatentDirichletAllocation(n_components = 1, random_state = 0)
  lda_train.fit(tfidf_vect)
  Result_G_n_text_lda_dict[G] = lda_train

진행중인 등급:  A+
진행중인 등급:  B
진행중인 등급:  B+
진행중인 등급:  A


In [63]:
total_feature_name=[]
# 나중에 test text에서 추출한 토픽과 유사도 비교를 위해서 리스트에 저장해 두기 위해 list 생성
total_str_list=[]

for G in ESG_G_n_text_dict.keys():
    lst=[]
    feature_name=Result_G_n_text_dict[G].get_feature_names()
    # 200 : 뽑을 토픽 키워드의 갯수로 해당 값도 실험을 통해 보편적으로 성능이 우수한 값 할당 
    save_topics(Result_G_n_text_lda_dict[G], feature_name, 300, lst)
    # esg 등급별로 뽑은 키워드 리스트에 순서대로 저장 (여기서는 A+,B,B+,A 순으로 저장 -> 21년 보고서 82개의 Total esg 등급이 저 4개로만 국한됨)
    total_str_list.insert(len(total_str_list),lst[0])

#### TEST 전처리 과정 

In [64]:
Test_df =pd.read_csv(path+"22Y_total_report.csv")
Test_df.drop(7, axis=0, inplace=True)
Test_df.reset_index(drop=True, inplace=True)
# df에 str형태로 전환이 되어서 저장이 되었기 때문에 str->list 그리고 그 안의 dict 형태로 읽기위해 변환
for i in range(len(Test_df)):
  Test_df['Text'].iloc[i] = ast.literal_eval(Test_df['Text'].iloc[i])

  # report 이릉에서 회사명만 추출해서 다시 name 변수로 저장 
  r_name = Test_df['Name'].iloc[i] 
  if r_name.endswith('_kr.pdf'):
    Test_df['Name'].iloc[i] = r_name.replace('_kr.pdf', '')
  else:
    Test_df['Name'].iloc[i] = r_name.replace('.pdf', '')
  
test_report_name_list = list(Test_df['Name'])

In [65]:
final_test_result_dict = {}

# train과 같은 과정 
for i in range(len(test_report_name_list)):
    page_word = []
    word_len = []
    page_word_len = []
    result_list = []
    page_text = Test_df['Text'].iloc[i][0]

    for page_num in range(len(page_text)):
      page_word.append([])
      page_word_len.append([])
      split_temp = re.split('[\n|\n\n]', page_text[page_num])
      for word in split_temp:
          if word != '':
              page_word[page_num].append(word) 
              page_word_len[page_num].append(len(word))
              word_len.append(len(word))
                
    mean_word = sum(word_len)/len(word_len)

    for page_num in range(len(page_text)):
        page_result = []
        for index in range(len(page_word_len[page_num])):
            len_w = page_word_len[page_num][index]
            if len_w > mean_word:
                page_result.append(page_word[page_num][index])
        result_list.append(page_result)
    final_test_result_dict[test_report_name_list[i]]=result_list
  

In [66]:
final_test_sumtext_dict = {}
for i in range(len(test_report_name_list)):
  file_name = test_report_name_list[i]
  text_list = final_test_result_dict[file_name]
  result_str = ''
  for texts in text_list:
    for sen in texts:
      result_str += sen
  result_str = clean_text(result_str)
  final_test_sumtext_dict[file_name] = result_str

In [67]:
pre_name_l = list(final_test_sumtext_dict.keys())
pre_text_l = list(final_test_sumtext_dict.values())
pre_df = pd.DataFrame({'Name':pre_name_l,'Text':pre_text_l})

In [68]:
# 미래에셋에서 준 Test data에 해당하는 기업정보 데이터 불러와서 전처리한 Text가 있는 데이터 프레임과 merge 
Test_com_df=pd.read_csv(path+"Test_data.csv")
final_Test_df = pd.merge(Test_com_df, pre_df, left_on='종목명',right_on='Name')

#### Test input 벡터화 그리고 LDA 적용 후 21년 esg별로 분할된 보고서의 키워드들이 저장된 리스트(total_str_list) 와 유사도 분석 후 가장 높은 index 그리고 각 등급별 코사인 유사도 저장

In [69]:
result_label = []
cos_result_list = []
tfidf_vect_test = []
for i in range(len(final_Test_df)):
  print("현재 진행중인 문서 번호: ",i)
  input_text = final_Test_df['Text'].iloc[i]
  input_text = tokenize(input_text)

  tfidf_vect=TfidfVectorizer(ngram_range=(1,2),max_df=500)
  tfidf_vect_test=tfidf_vect.fit_transform(input_text)
  feature_names_test = tfidf_vect.get_feature_names()
  
  # lda 모델에 입력 
  lda_test = LatentDirichletAllocation(n_components = 1, random_state = 0)
  lda_test.fit(tfidf_vect_test)

  test_list = []
  # 5000또한 실험으로 보편적으로 잘나오는 값 선정 
  save_topics(lda_test, feature_names_test, 5000, test_list)
  test_list=test_list[0]

  lst1 = []
  lst1.append(test_list)
  for j in range(len(total_str_list)):
      lst1.append(total_str_list[j])

  tfidf_vect_simple = TfidfVectorizer()
  feature_vect_simple = tfidf_vect_simple.fit_transform(lst1)

  cosine_similarity1 = cosine_similarity(feature_vect_simple[0], feature_vect_simple)
  cos_result_list.append(cosine_similarity1.tolist())
  result=cosine_similarity1.tolist()[0]

  del result[0]
  max_index=result.index(max(result))
  result_label.append(max_index)
  
# 결과물 label 저장 
final_Test_df['Result'] = result_label

현재 진행중인 문서 번호:  0
현재 진행중인 문서 번호:  1
현재 진행중인 문서 번호:  2
현재 진행중인 문서 번호:  3
현재 진행중인 문서 번호:  4
현재 진행중인 문서 번호:  5
현재 진행중인 문서 번호:  6
현재 진행중인 문서 번호:  7
현재 진행중인 문서 번호:  8
현재 진행중인 문서 번호:  9
현재 진행중인 문서 번호:  10
현재 진행중인 문서 번호:  11
현재 진행중인 문서 번호:  12
현재 진행중인 문서 번호:  13
현재 진행중인 문서 번호:  14
현재 진행중인 문서 번호:  15
현재 진행중인 문서 번호:  16
현재 진행중인 문서 번호:  17
현재 진행중인 문서 번호:  18
현재 진행중인 문서 번호:  19
현재 진행중인 문서 번호:  20
현재 진행중인 문서 번호:  21
현재 진행중인 문서 번호:  22
현재 진행중인 문서 번호:  23
현재 진행중인 문서 번호:  24
현재 진행중인 문서 번호:  25
현재 진행중인 문서 번호:  26
현재 진행중인 문서 번호:  27


In [70]:
# 아래 코드는 21년 82개의 보고서 기준으로 만들어졌기 때문에 이후 train 보고서 추가시에 등급할당에 대한 코드 수정이 필요 
for i in range(len(final_Test_df)):
   if final_Test_df['Result'].iloc[i] == 0:
     final_Test_df['Result'].iloc[i] = 'A+'
   elif final_Test_df['Result'].iloc[i] == 1:
     final_Test_df['Result'].iloc[i] = 'B'
   elif final_Test_df['Result'].iloc[i] == 2:
     final_Test_df['Result'].iloc[i] = 'B+'
   elif final_Test_df['Result'].iloc[i] == 3:
     final_Test_df['Result'].iloc[i] = 'A'

# 27개의 Test report의 22년 total_esg_score
final_Test_df['Real_label'] =['B+','B+','A+','A','A','A','A','B+','A','A','A','A','A','A','A','A','B','A','A','B','A','A','A','A','B+','B','B+','A']

In [71]:
# 보고서의 total_esg 등급을 정확하게 맞춘 갯수 
print(sum(final_Test_df['Result']==final_Test_df['Real_label']))
print("정확도: ",(sum(final_Test_df['Result']==final_Test_df['Real_label'])/len(final_Test_df))*100,"%")


18
정확도:  64.28571428571429 %


In [72]:
final_Test_df

,종목코드,종목명,시장,Name,Text,Result,Real_label
0,5930,삼성전자,KOSPI,삼성전자,삼성전자 지속가능경영보고서는 로 제작되었으며 독자의 이해를 돕기 위해 관련 웹페이...,A,B+
1,51910,LG화학,KOSPI,LG화학,2021 세계적흐름...,B+,B+
2,270,기아,KOSPI,기아,2022는 모빌리티를 통하여 세상에 긍정적인 변화를 만들어가는 움직임을 뜻하는...,A,A+
3,35720,카카오,KOSPI,카카오,1 2 카카오는 더 나은 세상을 만들기 위한 회사의 약속과 책임 활동을...,A,A
4,32830,삼성생명,KOSPI,삼성생명,2022 2 2022 발행처 삼성생명보험주식...,A,A
5,18260,삼성에스디에스,KOSPI,삼성에스디에스,삼성 지속가능경영보고서 2022 삼성는 경영활동을 통해 창출되는 경제 사...,A,A
6,3490,대한항공,KOSPI,대한항공,대한항공은 환경 사회 지배구조 측면에서 지속가능한 내일을 위한 환경경영 수송보국 ...,A,A
7,100,유한양행,KOSPI,유한양행,20212022 지금도 국민과 함께 자라고 있습니다유한양행은 환경 사회 지배구...,A,B+
8,139480,이마트,KOSPI,이마트,본 보고서는 이마트의 첫 번째 지속가능경영보고서입니다 본 보고서에는 지속가능...,A,A
9,42660,대우조선해양,KOSPI,대우조선해양,대우조선해양 통합보고서 2022 깨끗한 지구 ...,B,A
